In [1]:
import pandas as pd
import numpy as np

###### Tomando en cuenta las conclusiones de Tuneo_XGB_A tomo las siguientes consideraciones

In [2]:
col_importancia_1 = ["ciudad", "metroscubiertos", "metrostotales"]
col_importancia_2 = ["fecha_n", "provincia", "tipodepropiedad", "antiguedad"]
col_importancia_3 = ["habitaciones", "banos", "garages"]
col_importancia_4 = ["piscina", "gimnasio", "escuelascercanas", "centroscomercialescercanos", "usosmultiples"]

#### Voy a usar los siguientes encoders:
    -TargetEncoder
    -One hot encoder (para los mas importantes)
    -Count Encoding
    -CatBoost Encoding

In [3]:
from category_encoders import TargetEncoder, CountEncoder, CatBoostEncoder

In [4]:
def target_encoding(datos, precio, cols):
    codificador = TargetEncoder(cols=cols)
    return codificador.fit_transform(datos, precio)

def one_hot_encoding(datos, precio, feature_ohe, features):
    #No hago one hot de todo porque queda muy grande el dataframe
    df_ohe = datos[feature_ohe]
    df_1 = datos.drop(feature_ohe, axis=1)
    df_ohe = pd.get_dummies(df_ohe)
    
    df_1 = target_encoding(df_1, precio, features)
    
    df_respuesta = df_1.join(df_ohe)
    return df_respuesta

def count_encoding(df, col_categ, col_datos):
    codificador = CountEncoder()
    cod = codificador.fit_transform(df[col_categ])
    return df[col_datos].join(cod.add_suffix("_count"))
    
def catboost_encoding(datos, precio, cols):
    codificador = CatBoostEncoder(cols=cols)
    return codificador.fit_transform(datos, precio)

#### Voy a dar la opcion de eliminar o no los features de importancia 3

In [5]:
def eliminar_imp_3(df):
    return df.drop(col_importancia_3, axis=1)

#### Preparo el dataframe para tunear

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
#TODO: Hacer de esto una funcion o guardarlo hecho como CSV
#Levanto el df
train = pd.read_csv("sets_de_datos/train.csv", index_col=0)

#Convierto la fecha a numero
train["fecha"] = pd.to_datetime(train["fecha"])
train["fecha_n"] = train["fecha"].dt.year * 10000 + train["fecha"].dt.month * 100 + train["fecha"].dt.day

#Borro columnas
col_borrar = ["direccion", "lat", "lng", "titulo", "descripcion", "idzona", "fecha"]
train_1 = train.drop(col_borrar, axis=1)

#Borro las columnas de menor importancia
train_1 = train_1.drop(col_importancia_4, axis=1)

#Imputo los nans
train_1 = train_1.fillna(value = {'tipodepropiedad' : 'nan', 'provincia' : 'nan', 'ciudad' : 'nan'})

for c in ["antiguedad", "metrostotales", "metroscubiertos"]:
    imputer = SimpleImputer(strategy="mean")
    train_1[c] = imputer.fit_transform(train_1[[c]])

for c in ["habitaciones", "banos", "garages"]:
    imputer = SimpleImputer(strategy="constant", fill_value=0)
    train_1[c] = imputer.fit_transform(train_1[[c]])

In [8]:
datos = train_1.drop(["precio"], axis=1)
precio = train_1["precio"]

COL_CATEGORICAS = ["provincia", "ciudad", "tipodepropiedad"]
def col_no_categoricos(df):
    return df.dtypes[df.dtypes != "object"].index

#### Creo el espacio de tuneo

In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [10]:
#Constantes del espacio de tuneo
TARGET = 0
OHE = 1
COUNT = 2
CATBOOST = 3

ELIMINAR_IMP_3 = 0
NO_ELIMINAR_IMP_3 = 1

In [11]:
espacio_xgb = {
    "objective" : "reg:squarederror",
    "encoder" : hp.choice("encoder", [TARGET, OHE, COUNT, CATBOOST]),
    "importancia_3" : hp.choice("importancia_3", [ELIMINAR_IMP_3, NO_ELIMINAR_IMP_3]),
    "n_estimators" : hp.choice("n_estimators", range(1000, 2001)),
    "min_child_weight" : hp.choice("min_child_weight", range(1,6)),
    "max_depth" : hp.choice("max_depth", range(8, 16)),
    "learning_rate" : hp.uniform("learning_rate", 0.01, 0.1),
    "reg_lambda" : hp.uniform("reg_lambda", 0.03, 0.3),
    "gamma" : hp.uniform("gamma", 0, 0.5),
    "colsample_bytree" : hp.uniform("colsample_bytree", 0.5, 1),
    "subsample" : hp.uniform("subsample", 0.5, 1)
}

#### Funcion del modelo

In [12]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
import xgboost as xgb

In [13]:
def hyperopt_train_test(params, datos, precio):
    datos_ = datos
    #================FEATURES===================
    if params["importancia_3"] == ELIMINAR_IMP_3:
        datos_ = eliminar_imp_3(datos_)
    
    #===============ENCODING============
    if params["encoder"] == TARGET:
        datos_ = target_encoding(datos_, precio, COL_CATEGORICAS)
        
    if params["encoder"] == OHE:
        datos_ = one_hot_encoding(datos_, precio, ["ciudad"], ["tipodepropiedad", "provincia"])
        
    if params["encoder"] == COUNT:
        datos_ = count_encoding(datos_, COL_CATEGORICAS, col_no_categoricos(datos_))
        
    if params["encoder"] == CATBOOST:
        datos_ = catboost_encoding(datos_, precio, COL_CATEGORICAS)
    
    
    modelo = XGBRegressor(**params)
    return cross_val_score(modelo, datos_, precio).mean()

In [14]:
def f(params):
    acc = hyperopt_train_test(params, datos, precio)
    return {'loss': -acc, 'status': STATUS_OK}

#### Tuneo

In [15]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [16]:
trials = Trials()

In [18]:
best = fmin(f, espacio_xgb, algo=tpe.suggest, max_evals=2, trials=trials)

100%|██████████| 2/2 [34:09<00:00, 1024.54s/it, best loss: -0.7965954181618882]


In [19]:
best

{'colsample_bytree': 0.5522426459530355,
 'encoder': 0,
 'gamma': 0.049417927153982966,
 'importancia_3': 1,
 'learning_rate': 0.028491939247932148,
 'max_depth': 3,
 'min_child_weight': 0,
 'n_estimators': 926,
 'reg_lambda': 0.18106491312186662,
 'subsample': 0.7925982324384148}

In [21]:
#Hice un ejemplo con 2 evaluaciones de juguete, uno mas grande me daba problemas de memoria
#corri uno un poco mas grande con google colab